# Neuron Network - Lab

### Part 1: Load  data

Import "bank-data.csv"

In [136]:
import pandas as pd
bankData = pd.read_csv('bank-data.csv', sep = ';')
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


### Part 2: Preprocess data

Preprocess the dataset as you have done before

#### 2.1 Binary encoding

Use LabelEncoder to encode the following columns:
- y
- default
- housing
- loan

In [137]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#example
bankData['y'] = le.fit_transform(bankData['y'])
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


#### 2.2 Convert categorical variables into dummy columns

(1) Use pd.get_dummies to convert the following categorical variales into dummy columns
- job
- maritial
- education
- contact
- month
- poutcome

(2) Drop columns that have been converted

In [138]:
#Encode the remaining columns
bankData['housing'] = le.fit_transform(bankData['housing'])
bankData['default'] = le.fit_transform(bankData['default'])
bankData['loan'] = le.fit_transform(bankData['loan'])
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0


In [139]:
#example
bankData = pd.concat([bankData,pd.get_dummies(bankData['job'],prefix='job')],axis=1)
bankData = bankData.drop(columns=['job'])
bankData.head()

,age,marital,education,default,balance,housing,loan,contact,day,month,...,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown
0,30,married,primary,0,1787,0,0,cellular,19,oct,...,0,0,0,0,0,0,0,0,1,0
1,33,married,secondary,0,4789,1,1,cellular,11,may,...,0,0,0,0,0,1,0,0,0,0
2,35,single,tertiary,0,1350,1,0,cellular,16,apr,...,0,0,1,0,0,0,0,0,0,0
3,30,married,tertiary,0,1476,1,1,unknown,3,jun,...,0,0,1,0,0,0,0,0,0,0
4,59,married,secondary,0,0,1,0,unknown,5,may,...,0,0,0,0,0,0,0,0,0,0


In [140]:
bankData = pd.concat([bankData,pd.get_dummies(bankData['marital'],prefix='marital')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['education'],prefix='education')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['contact'],prefix='contact')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['month'],prefix='month')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['poutcome'],prefix='poutcome')],axis=1)

bankData = bankData.drop(columns=['marital', 'education', 'contact', 'month', 'poutcome'])

In [141]:
bankData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 49 columns):
age                    4521 non-null int64
default                4521 non-null int64
balance                4521 non-null int64
housing                4521 non-null int64
loan                   4521 non-null int64
day                    4521 non-null int64
duration               4521 non-null int64
campaign               4521 non-null int64
pdays                  4521 non-null int64
previous               4521 non-null int64
y                      4521 non-null int64
job_admin.             4521 non-null uint8
job_blue-collar        4521 non-null uint8
job_entrepreneur       4521 non-null uint8
job_housemaid          4521 non-null uint8
job_management         4521 non-null uint8
job_retired            4521 non-null uint8
job_self-employed      4521 non-null uint8
job_services           4521 non-null uint8
job_student            4521 non-null uint8
job_technician         4521 non-n

#### 2.3 Train/Test separation

Perform hold-out method
- 60% training set
- 40% testing set

In [142]:
bankData_train = bankData.sample(frac = 0.6)
bankData_test = bankData.drop(bankData_train.index)
print(pd.crosstab(bankData_train['y'],columns = 'count'))
print(pd.crosstab(bankData_test['y'],columns = 'count'))

col_0  count
y           
0       2382
1        331
col_0  count
y           
0       1618
1        190


##### X/y separation

In [143]:
bankData_train_y = bankData_train['y']
bankData_train_X = bankData_train.copy()
del bankData_train_X['y']

bankData_test_y = bankData_test['y']
bankData_test_X = bankData_test.copy()
del bankData_test_X['y']

#### 2.4 Feature Scaling

It is always a good practice to scale the features so that all of them can be uniformly evaluated

In [144]:
from sklearn import preprocessing

standard_scaler = preprocessing.StandardScaler()
train_X_scaled_s = pd.DataFrame(standard_scaler.fit_transform(bankData_train_X), columns=bankData_train_X.columns)
test_X_scaled_s = pd.DataFrame(standard_scaler.fit_transform(bankData_test_X), columns=bankData_train_X.columns)

min_max_scaler = preprocessing.MinMaxScaler()
train_X_scaled_m = pd.DataFrame(min_max_scaler.fit_transform(bankData_train_X),columns=bankData_train_X.columns)
test_X_scaled_m = pd.DataFrame(min_max_scaler.fit_transform(bankData_test_X),columns=bankData_train_X.columns)

In [145]:
train_X_scaled_s.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0.757867,-0.128396,-0.506384,-1.145807,-0.419507,-1.334332,-0.125078,-0.256160,-0.409033,-0.322386,...,-0.374053,-0.105743,1.511457,-0.310813,-0.143848,-0.110966,-0.355237,-0.218849,-0.162742,0.474163
1,0.472615,-0.128396,0.798077,0.872748,-0.419507,-0.233309,1.122996,-0.256160,-0.409033,-0.322386,...,-0.374053,-0.105743,1.511457,-0.310813,-0.143848,-0.110966,-0.355237,-0.218849,-0.162742,0.474163
2,0.948034,-0.128396,-0.362465,0.872748,2.383750,0.011363,-0.647615,-0.576567,-0.409033,-0.322386,...,-0.374053,-0.105743,-0.661613,-0.310813,6.951782,-0.110966,-0.355237,-0.218849,-0.162742,0.474163
3,1.233286,-0.128396,-0.524374,-1.145807,-0.419507,0.011363,-0.561152,0.705060,-0.409033,-0.322386,...,-0.374053,-0.105743,-0.661613,-0.310813,-0.143848,-0.110966,-0.355237,-0.218849,-0.162742,0.474163
4,-0.192971,-0.128396,-0.124385,0.872748,2.383750,-0.844988,0.085440,3.268314,-0.409033,-0.322386,...,-0.374053,-0.105743,-0.661613,-0.310813,-0.143848,-0.110966,-0.355237,-0.218849,-0.162742,0.474163


In [146]:
train_X_scaled_m.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0.447761,0.0,0.072314,0.0,0.0,0.133333,0.075828,0.023256,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.402985,0.0,0.188071,1.0,0.0,0.433333,0.185762,0.023256,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.477612,0.0,0.085085,1.0,1.0,0.500000,0.029801,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.522388,0.0,0.070718,0.0,0.0,0.500000,0.037417,0.093023,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.298507,0.0,0.106212,1.0,1.0,0.266667,0.094371,0.279070,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Artificial Neural Network : sklearn

### Part 3: Train a model

In [147]:
from sklearn.neural_network import MLPClassifier  
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)  
mlp.fit(train_X_scaled_s, bankData_train_y) 

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 10, 10), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

### Part 4: Model Evaluation

Evaluation metrics
- confusion metrix
- accuracy
- precision, recall, f1-score

In [148]:
from sklearn.metrics import classification_report

# predict
pred = mlp.predict(bankData_test_X)

# evaluate
print(pd.crosstab(bankData_test_y, pred))
print(classification_report(bankData_test_y, pred))

col_0     0    1
y               
0      1495  123
1       171   19
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1618
           1       0.13      0.10      0.11       190

    accuracy                           0.84      1808
   macro avg       0.52      0.51      0.51      1808
weighted avg       0.82      0.84      0.83      1808



### Part 5: Model tuning

#### Note:

After building the classifier, try answering the following questions.

1. What is the Accuracy Score?
2. If you change your preprosessing method, can you improve the model?
3. If you change your parameters setting, can you improve the model?

In [149]:
from sklearn.model_selection import train_test_split

# x,y split
features = bankData.drop(['y'], axis=1)
label = bankData['y']
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=0)


In [150]:
# scale data
minMaxScaler = preprocessing.MinMaxScaler()
x_train_scaled = minMaxScaler.fit_transform(x_train)
x_test_scaled = minMaxScaler.fit_transform(x_test)

In [151]:
# train model
nn1 = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)  
nn1.fit(x_train_scaled, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 10, 10), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [152]:
# predict
tuned_pred = nn1.predict(x_test_scaled)

# evaluate
print(pd.crosstab(y_test, tuned_pred))
print(classification_report(y_test, tuned_pred))

col_0     0   1
y              
0      1117  73
1        76  91
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1190
           1       0.55      0.54      0.55       167

    accuracy                           0.89      1357
   macro avg       0.75      0.74      0.74      1357
weighted avg       0.89      0.89      0.89      1357



<b>Q1:</b> What is the Accuracy Score? <br>
<b>A1:</b> Accuracy score of changed preprocessing process is about 0.87.

<b>Q2:</b> If you change your preprosessing method, can you improve the model?  <br>
<b>A2:</b> Yes. As the experiment above, I have changed the train/test data size from 60% train 40% test to 70% train and 30% test data. Moreover, I have changed the data scaling method from StandardScaler to MinMaxScaler and try with the same parameter setting model again, the result is more satisfy, the accuracy increase from 0.84 to 0.89.

In [181]:
nn2 = MLPClassifier(hidden_layer_sizes=(10, 5, 10), max_iter=15000)  
nn2.fit(x_train_scaled, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 5, 10), learning_rate='constant',
              learning_rate_init=0.001, max_iter=15000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [182]:
# predict
tuned_pred = nn2.predict(x_test_scaled)

# evaluate
print(pd.crosstab(y_test, tuned_pred))
print(classification_report(y_test, tuned_pred))

col_0     0   1
y              
0      1119  71
1        85  82
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      1190
           1       0.54      0.49      0.51       167

    accuracy                           0.89      1357
   macro avg       0.73      0.72      0.72      1357
weighted avg       0.88      0.89      0.88      1357



<b>Q3:</b> If you change your parameters setting, can you improve the model? <br>
<b>A3:</b> Yes. As the above cell, I preserved the same data preprocessing process but changed the amount of node of the middle layer inside the hidden layer from 10 to 5 nodes these changing result more 0.05 accuracy from the prior model.

## Artificial Neural Network : keras

Fitting a logistic regression model

### Part 3: Train a model

In [183]:
import keras
from keras import models
from keras import layers

In [184]:
train_X_scaled_s.shape

(2713, 48)

In [185]:
nn = models.Sequential()
nn.add(layers.Dense(48,activation = 'linear',input_shape=(None,48)))
nn.add(layers.Dense(1,activation = 'sigmoid'))

In [186]:
nn.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [187]:
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, None, 48)          2352      
_________________________________________________________________
dense_10 (Dense)             (None, None, 1)           49        
Total params: 2,401
Trainable params: 2,401
Non-trainable params: 0
_________________________________________________________________


In [188]:
import numpy as np
X_train_add = np.expand_dims(train_X_scaled_s, axis=0)
y_train_add = np.expand_dims(bankData_train_y, axis=0)
y_train_add = np.expand_dims(y_train_add, axis=2)

In [189]:
y_train_add.shape

(1, 2713, 1)

In [190]:
history = nn.fit(X_train_add,y_train_add,epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 431ms/step - loss: 0.9202 - accuracy: 0.5009
Epoch 2/100
1/1 [==============================] - 0s 9ms/step - loss: 0.9104 - accuracy: 0.5042
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 0.9007 - accuracy: 0.5072
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 0.8913 - accuracy: 0.5127
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8820 - accuracy: 0.5164
Epoch 6/100
1/1 [==============================] - 0s 8ms/step - loss: 0.8730 - accuracy: 0.5212
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8641 - accuracy: 0.5223
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 0.8553 - accuracy: 0.5260
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8467 - accuracy: 0.5297
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8383 - accuracy: 0.5345
Epoch 11/100
1/1 [=========

Epoch 85/100
1/1 [==============================] - 0s 4ms/step - loss: 0.4893 - accuracy: 0.8194
Epoch 86/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4869 - accuracy: 0.8212
Epoch 87/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4845 - accuracy: 0.8238
Epoch 88/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4821 - accuracy: 0.8264
Epoch 89/100
1/1 [==============================] - 0s 4ms/step - loss: 0.4798 - accuracy: 0.8286
Epoch 90/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4775 - accuracy: 0.8327
Epoch 91/100
1/1 [==============================] - 0s 4ms/step - loss: 0.4752 - accuracy: 0.8352
Epoch 92/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4730 - accuracy: 0.8356
Epoch 93/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4708 - accuracy: 0.8360
Epoch 94/100
1/1 [==============================] - 0s 4ms/step - loss: 0.4686 - accuracy: 0.8367
Epoch 95/100
1/1 [==

### Part 4: Model Evaluation

In [191]:
X_test_add = np.expand_dims(test_X_scaled_s, axis=0)
y_test_add = np.expand_dims(bankData_test_y, axis=0)
y_test_add = np.expand_dims(y_test_add, axis=2)

In [192]:
test_loss, test_acc = nn.evaluate(X_test_add, y_test_add)
print('Test Loss: %s\nTest Accuracy: %s' % (test_loss,test_acc))

1/1 [==============================] - 0s 144ms/step
Test Loss: 0.44879457354545593
Test Accuracy: 0.8506637215614319


### Part 5: Model tuning

#### Note:

After building the classifier, try answering the following questions.

1. What is the Accuracy Score?
2. If you change your preprosessing method, can you improve the model?
3. If you change your parameters setting, can you improve the model?

In [193]:
from sklearn.model_selection import train_test_split

# x,y split
features = bankData.drop(['y'], axis=1)
label = bankData['y']
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=0)

In [194]:
# scale data
standardScaler = preprocessing.StandardScaler()
x_train_scaled = standardScaler.fit_transform(x_train)
x_test_scaled = standardScaler.fit_transform(x_test)

In [195]:
X_train_add = np.expand_dims(x_train_scaled, axis=0)
y_train_add = np.expand_dims(y_train, axis=0)
y_train_add = np.expand_dims(y_train_add, axis=2)

In [196]:
history = nn.fit(X_train_add,y_train_add,epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 6ms/step - loss: 0.4533 - accuracy: 0.8496
Epoch 2/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4512 - accuracy: 0.8499
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4492 - accuracy: 0.8508
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4472 - accuracy: 0.8511
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4452 - accuracy: 0.8515
Epoch 6/100
1/1 [==============================] - 0s 7ms/step - loss: 0.4432 - accuracy: 0.8530
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4413 - accuracy: 0.8540
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4394 - accuracy: 0.8549
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 0.4375 - accuracy: 0.8562
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4356 - accuracy: 0.8571
Epoch 11/100
1/1 [===========

Epoch 85/100
1/1 [==============================] - 0s 4ms/step - loss: 0.3427 - accuracy: 0.8948
Epoch 86/100
1/1 [==============================] - 0s 5ms/step - loss: 0.3419 - accuracy: 0.8948
Epoch 87/100
1/1 [==============================] - 0s 4ms/step - loss: 0.3411 - accuracy: 0.8954
Epoch 88/100
1/1 [==============================] - 0s 5ms/step - loss: 0.3403 - accuracy: 0.8957
Epoch 89/100
1/1 [==============================] - 0s 4ms/step - loss: 0.3395 - accuracy: 0.8957
Epoch 90/100
1/1 [==============================] - 0s 4ms/step - loss: 0.3388 - accuracy: 0.8957
Epoch 91/100
1/1 [==============================] - 0s 5ms/step - loss: 0.3380 - accuracy: 0.8951
Epoch 92/100
1/1 [==============================] - 0s 4ms/step - loss: 0.3372 - accuracy: 0.8951
Epoch 93/100
1/1 [==============================] - 0s 4ms/step - loss: 0.3365 - accuracy: 0.8954
Epoch 94/100
1/1 [==============================] - 0s 5ms/step - loss: 0.3357 - accuracy: 0.8954
Epoch 95/100
1/1 [==

In [197]:
X_test_add = np.expand_dims(x_test_scaled, axis=0)
y_test_add = np.expand_dims(y_test, axis=0)
y_test_add = np.expand_dims(y_test_add, axis=2)

In [198]:
test_loss, test_acc = nn.evaluate(X_test_add, y_test_add)
print('Test Loss: %s\nTest Accuracy: %s' % (test_loss,test_acc))

1/1 [==============================] - 0s 3ms/step
Test Loss: 0.3324665129184723
Test Accuracy: 0.8813559412956238


<b>Q1:</b> What is the Accuracy Score? <br>
<b>A1:</b> Accuracy score of changed preprocessing process is about 0.87.

<b>Q2:</b> If you change your preprosessing method, can you improve the model? <br>
<b>A2:</b> Yes. As the experimental above, I changed the preprocessing process of train/test data from 60% train data 40% test to 70% train data 30% test data with the same scale method. The experiment result more 0.03 accuracy.

In [206]:
X_train_add = np.expand_dims(train_X_scaled_s, axis=0)
y_train_add = np.expand_dims(bankData_train_y, axis=0)
y_train_add = np.expand_dims(y_train_add, axis=2)

In [207]:
keras_nn = models.Sequential()
keras_nn.add(layers.Dense(48,activation = 'linear',input_shape=(None,48)))
keras_nn.add(layers.Dense(1,activation = 'sigmoid'))

In [208]:
keras_nn.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [209]:
keras_nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, None, 48)          2352      
_________________________________________________________________
dense_14 (Dense)             (None, None, 1)           49        
Total params: 2,401
Trainable params: 2,401
Non-trainable params: 0
_________________________________________________________________


In [210]:
history2 = keras_nn.fit(X_train_add,y_train_add,epochs=300)

Epoch 1/300
1/1 [==============================] - 0s 493ms/step - loss: 0.8566 - accuracy: 0.5131
Epoch 2/300
1/1 [==============================] - 0s 5ms/step - loss: 0.8487 - accuracy: 0.5182
Epoch 3/300
1/1 [==============================] - 0s 4ms/step - loss: 0.8410 - accuracy: 0.5216
Epoch 4/300
1/1 [==============================] - 0s 4ms/step - loss: 0.8334 - accuracy: 0.5260
Epoch 5/300
1/1 [==============================] - 0s 4ms/step - loss: 0.8259 - accuracy: 0.5297
Epoch 6/300
1/1 [==============================] - 0s 5ms/step - loss: 0.8187 - accuracy: 0.5315
Epoch 7/300
1/1 [==============================] - 0s 5ms/step - loss: 0.8115 - accuracy: 0.5396
Epoch 8/300
1/1 [==============================] - 0s 4ms/step - loss: 0.8045 - accuracy: 0.5444
Epoch 9/300
1/1 [==============================] - 0s 5ms/step - loss: 0.7976 - accuracy: 0.5477
Epoch 10/300
1/1 [==============================] - 0s 5ms/step - loss: 0.7909 - accuracy: 0.5518
Epoch 11/300
1/1 [=========

Epoch 85/300
1/1 [==============================] - 0s 4ms/step - loss: 0.5009 - accuracy: 0.7892
Epoch 86/300
1/1 [==============================] - 0s 4ms/step - loss: 0.4987 - accuracy: 0.7906
Epoch 87/300
1/1 [==============================] - 0s 4ms/step - loss: 0.4965 - accuracy: 0.7932
Epoch 88/300
1/1 [==============================] - 0s 4ms/step - loss: 0.4944 - accuracy: 0.7973
Epoch 89/300
1/1 [==============================] - 0s 4ms/step - loss: 0.4922 - accuracy: 0.8002
Epoch 90/300
1/1 [==============================] - 0s 4ms/step - loss: 0.4901 - accuracy: 0.8032
Epoch 91/300
1/1 [==============================] - 0s 4ms/step - loss: 0.4881 - accuracy: 0.8058
Epoch 92/300
1/1 [==============================] - 0s 4ms/step - loss: 0.4860 - accuracy: 0.8083
Epoch 93/300
1/1 [==============================] - 0s 4ms/step - loss: 0.4840 - accuracy: 0.8109
Epoch 94/300
1/1 [==============================] - 0s 5ms/step - loss: 0.4820 - accuracy: 0.8124
Epoch 95/300
1/1 [==

Epoch 168/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3791 - accuracy: 0.8850
Epoch 169/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3782 - accuracy: 0.8861
Epoch 170/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3772 - accuracy: 0.8872
Epoch 171/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3763 - accuracy: 0.8868
Epoch 172/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3754 - accuracy: 0.8872
Epoch 173/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3745 - accuracy: 0.8876
Epoch 174/300
1/1 [==============================] - 0s 4ms/step - loss: 0.3736 - accuracy: 0.8876
Epoch 175/300
1/1 [==============================] - 0s 6ms/step - loss: 0.3727 - accuracy: 0.8883
Epoch 176/300
1/1 [==============================] - 0s 6ms/step - loss: 0.3718 - accuracy: 0.8887
Epoch 177/300
1/1 [==============================] - 0s 4ms/step - loss: 0.3709 - accuracy: 0.8883
Epoch 178/

Epoch 251/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3230 - accuracy: 0.8946
Epoch 252/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3225 - accuracy: 0.8950
Epoch 253/300
1/1 [==============================] - 0s 4ms/step - loss: 0.3221 - accuracy: 0.8946
Epoch 254/300
1/1 [==============================] - 0s 4ms/step - loss: 0.3216 - accuracy: 0.8946
Epoch 255/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3212 - accuracy: 0.8946
Epoch 256/300
1/1 [==============================] - 0s 5ms/step - loss: 0.3207 - accuracy: 0.8946
Epoch 257/300
1/1 [==============================] - 0s 4ms/step - loss: 0.3203 - accuracy: 0.8938
Epoch 258/300
1/1 [==============================] - 0s 4ms/step - loss: 0.3198 - accuracy: 0.8935
Epoch 259/300
1/1 [==============================] - 0s 4ms/step - loss: 0.3194 - accuracy: 0.8935
Epoch 260/300
1/1 [==============================] - 0s 4ms/step - loss: 0.3189 - accuracy: 0.8942
Epoch 261/

In [211]:
X_test_add = np.expand_dims(test_X_scaled_s, axis=0)
y_test_add = np.expand_dims(bankData_test_y, axis=0)
y_test_add = np.expand_dims(y_test_add, axis=2)

In [212]:
test_loss, test_acc = keras_nn.evaluate(X_test_add, y_test_add)
print('Test Loss: %s\nTest Accuracy: %s' % (test_loss,test_acc))

1/1 [==============================] - 0s 150ms/step
Test Loss: 0.2970810830593109
Test Accuracy: 0.903761088848114


<b>Q3:</b> If you change your parameters setting, can you improve the model? <br>
<b>A3:</b> Yes. I changed the epoch from 100 to 300 and the model accuracy is increased from the prior model that gave averaged 0.85 accuracy to averaged 0.90 accuracy.